# Issue a Credential using BIP 322 Signatures

This notebook outlines how a Bitcoin address can be used to sign a verifiable credential using BIP 322 signatures explored in other notebooks. Initially, this will just look at using a bitcoin address as the issuer although eventually this should transition to a did:btcr (v2?)

## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and Digital Contract Design. Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.

In [3]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## Steps

1. Define VC
2. Canonicalise VC (Do we need this?)
3. BIP 322 sign VC payload
4. Attach signature to VC as proof

## 1. Define VC


Issuer needs to decide:

* Contents of credential
* Schema of credential
* Key pair and associated identifier they will be signing with
* Identifier to use to the credential subject


### 1.1. TODO: How do we identify the issuer

Should be use a did:btcr? Would need to be able to resolve this to a DID document and incombination with the proof data fetch the bitcoin address required to verify the signature.

In [4]:
unsigned_vc = {
  
  "@context": [
    "https://www.w3.org/2018/credentials/v1",
    "https://www.w3.org/2018/credentials/examples/v1"
  ],
  
  "id": "http://example.edu/credentials/1872",
  


  "type": ["VerifiableCredential", "AlumniCredential"],
  

  # TODO: How identifier issuer
  "issuer": "https://example.edu/issuers/565049",
  


  "issuanceDate": "2010-01-01T19:23:24Z",
  


  "credentialSubject": {
    

    # TODO: How identify credential subject
    "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
    

    "alumniOf": {
      "id": "did:example:c276e12ec21ebfeb1f712ebc6f1",
      "name": [{
        "value": "Example University",
        "lang": "en"
      }, {
        "value": "Exemple d'Université",
        "lang": "fr"
      }]
    }
  },
  


}

## 2. Canonicalize VC

Not exactly sure what this contains. Need a way to determinstically turn JSONLD VC object into a message to be signed.

I think only need to canonicalize if rendering data in JSONLD VC from other sources? RDF Dataset?

- https://json-ld.github.io/rdf-dataset-canonicalization/spec/
- https://www.w3.org/TR/vc-data-model/#json-ld

In [6]:
# Naive approach?

import json
from buidl.helper import str_to_bytes

vc_payload = json.dumps(unsigned_vc)
print(str_to_bytes(vc_payload))

b'{"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d\'Universit\\u00e9", "lang": "fr"}]}}}'


## 3. Sign VC

**Code for sign_message can be found in [message.py](../../edit/src/message.py)**

In [7]:
from buidl.ecc import PrivateKey
private_key_wif = "L4qmTC36US6tngrqbrW6vffZbxUVSfipQqXxdJw8S5HqyxNebaMP"
private_key = PrivateKey.parse(private_key_wif)

In [8]:
address = private_key.point.p2wpkh_script().address()
address

'bc1qz52z3pe9fg3qxv9n6yhxgj7rcn8wsvpq56v9ck'

In [9]:
from src.message import sign_message, MessageSignatureFormat

vc_sig = sign_message(MessageSignatureFormat.SIMPLE, private_key, address, vc_payload)
print(vc_sig)

AkgwRQIhAKgZ4oVu51lwBxQ+u0yz1xHBg60ovueTTvU8Wl5RkrhoAiAfY5WS+ghkFmQZzhLcGN6wSAjlQeLbBYgPi1Ve3hvnswEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x


## 4. Attach Signature to VC as proof

In the BIP322 Signature Suite (TBD) we would define three proof types?

* BIP322LegacySignature2022 
* BIP322SimpleSignature2022
* BIP322FullSignature2022

These types would define how the proof JSON should be structured and interpretted

In [10]:


proof = {

    "type": "BIP322SimpleSignature2022",
    


    "created": "2022-05-23T21:19:10Z",
    


    "proofPurpose": "assertionMethod",
    

    
    # This verification method would identify a UTXO on the bitcoin network?
    "verificationMethod": "https://example.edu/issuers/565049#key-1",
    


    "bip322_sig": vc_sig
}

In [12]:
signed_vc = unsigned_vc
signed_vc["proof"] = proof
print(json.dumps(signed_vc))

{"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d'Universit\u00e9", "lang": "fr"}]}}, "proof": {"type": "BIP322SimpleSignature2022", "created": "2022-05-23T21:19:10Z", "proofPurpose": "assertionMethod", "verificationMethod": "https://example.edu/issuers/565049#key-1", "bip322_sig": "AkgwRQIhAKgZ4oVu51lwBxQ+u0yz1xHBg60ovueTTvU8Wl5RkrhoAiAfY5WS+ghkFmQZzhLcGN6wSAjlQeLbBYgPi1Ve3hvnswEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x"}}
